In [1]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.cross_validation import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import scipy.stats as stats
import seaborn as sns

%pwd
#os.chdir('C:\\Users\\Prudhvi\\Desktop\\Prudhvi\\Data Science\\Data')
os.chdir('C:\\Users\\KASTU1\\Desktop\\Analytics Path\\R\\Data')
%pwd

C:\Users\KASTU1\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


'C:\\Users\\KASTU1\\Desktop\\Analytics Path\\R\\Data'

In [154]:
churn=pd.read_csv("Teleco_Cust_Attr_1.csv",sep=',')

In [155]:
churn.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [156]:
## Data Type Conversion
churn["SeniorCitizen"]=churn["SeniorCitizen"].astype(object)

## Replacing few values 
churn.loc[churn.OnlineSecurity=="No internet service","OnlineSecurity"]="No"
churn.loc[churn.OnlineBackup=="No internet service","OnlineBackup"]="No"
churn.loc[churn.DeviceProtection=="No internet service","DeviceProtection"]="No"
churn.loc[churn.TechSupport=="No internet service","TechSupport"]="No"
churn.loc[churn.StreamingTV=="No internet service","StreamingTV"]="No"
churn.loc[churn.StreamingMovies=="No internet service","StreamingMovies"]="No"
churn.loc[churn.MultipleLines=="No phone service","MultipleLines"]="No"

In [157]:
from sklearn.preprocessing import LabelEncoder
lc = LabelEncoder()
for i in churn.columns:
    if(churn[i].dtype.name=="object"):
        churn[i] = lc.fit_transform(churn[i])
        churn[i] = churn[i].astype("category")

In [158]:
## Dropping few columns
churn.drop("customerID", axis = 1, inplace=True)
churn.drop("gender",axis=1,inplace=True)
#churn.drop("PhoneService",axis=1,inplace=True)

In [159]:
churn['plan']='99999999'
for i in range(len(churn)):
    a=churn['MultipleLines'][i]
    c=churn['OnlineSecurity'][i]
    d=churn['OnlineBackup'][i]
    e=churn['DeviceProtection'][i]
    f=churn['TechSupport'][i]
    g=churn['StreamingTV'][i]
    h=churn['StreamingMovies'][i]
    churn['plan'][i]=str(a)+str(c)+str(d)+str(e)+str(f)+str(g)+str(h)

C:\Users\KASTU1\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [160]:
concat_columns=['MultipleLines','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']
for i in concat_columns:
    churn.drop(i,axis=1,inplace=True)

In [161]:
churn['plan']=churn['plan'].astype('category')

In [162]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
cm_train=confusion_matrix(churn['Churn'],churn['Churn'])
print(cm_train)
classificaiton_report_train=classification_report(churn['Churn'],churn['Churn'])
print(classificaiton_report_train)

[[5174    0]
 [   0 1869]]
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5174
          1       1.00      1.00      1.00      1869

avg / total       1.00      1.00      1.00      7043



In [163]:
## Feature Engineering
#churn.insert(0, "Charges", 0)
total_rows = churn['Churn'].count()
for i in range(0,total_rows):
    churn.loc[i,"Charges"]=churn.loc[i,"tenure"]*churn.loc[i,"MonthlyCharges"]

In [63]:
#churn.to_csv('telco_charges.csv')

In [164]:
churn.drop("TotalCharges", axis = 1,  inplace=True)
churn.drop("tenure",axis=1,inplace=True)
churn.drop("MonthlyCharges",axis=1,inplace=True)

In [165]:
churn.head()
churn.dtypes

SeniorCitizen       category
Partner             category
Dependents          category
PhoneService        category
InternetService     category
Contract            category
PaperlessBilling    category
PaymentMethod       category
Churn               category
plan                category
Charges              float64
dtype: object

In [166]:
from sklearn.preprocessing import OneHotEncoder
oe = OneHotEncoder()
churn_cat=[]
churn_ncat=[]
for i in churn.columns:
    if(churn[i].dtype.name=="category" and i!='Churn' ):
        if(len(churn[i].unique()) > 1):
            churn_cat.append(i) 
        else:
            churn_ncat.append(i)
    else:
        churn_ncat.append(i)


churn_cat = churn[churn_cat]
churn_ncat = churn[churn_ncat]
print("***** Categorical Columns*****")
print(churn_cat.columns)
print("***** Non Categorical Columns*****")
print(churn_ncat.columns)

churn_cat = pd.get_dummies(churn_cat,drop_first=True)
fn_dataset = pd.concat([churn_cat,churn_ncat],axis=1)

***** Categorical Columns*****
Index(['SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
       'InternetService', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'plan'],
      dtype='object')
***** Non Categorical Columns*****
Index(['Churn', 'Charges'], dtype='object')


In [167]:
fn_dataset = pd.concat([churn_cat,churn_ncat],axis=1)
Y=fn_dataset['Churn']
Y=pd.DataFrame(Y)
type(Y)
fn_dataset.drop('Churn',axis=1,inplace=True)
fn_dataset=pd.concat([fn_dataset,Y],axis=1)
fn_dataset.head()

,SeniorCitizen_1,Partner_1,Dependents_1,PhoneService_1,InternetService_1,InternetService_2,Contract_1,Contract_2,PaperlessBilling_1,PaymentMethod_1,...,plan_1111000,plan_1111001,plan_1111010,plan_1111011,plan_1111100,plan_1111101,plan_1111110,plan_1111111,Charges,Churn
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,29.85,0
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1936.30,0
2,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,107.70,1
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1903.50,0
4,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,141.40,1


In [168]:
#churn_cat = pd.get_dummies(fn_dataset,drop_first=True)
X = fn_dataset.iloc[:,:-1]
Y = fn_dataset.iloc[:,-1:]
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 0)

In [169]:
X_train.head()

,SeniorCitizen_1,Partner_1,Dependents_1,PhoneService_1,InternetService_1,InternetService_2,Contract_1,Contract_2,PaperlessBilling_1,PaymentMethod_1,...,plan_1110111,plan_1111000,plan_1111001,plan_1111010,plan_1111011,plan_1111100,plan_1111101,plan_1111110,plan_1111111,Charges
3296,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1183.20
6397,1,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3523.50
6043,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,120.45
5309,0,1,0,1,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1235.25
3000,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1013.40


In [171]:
y_train.head()

,Churn
3296,0
6397,0
6043,1
5309,0
3000,1


In [145]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(penalty='l2',C=5)
print(classifier)
##Fitting the model
result=classifier.fit(X_train, y_train)

#Prediction
result_train=result.predict_proba(X_train)
result_test=result.predict(X_train)

#Converting result to a data frame
result_train=pd.DataFrame(result_train)
result_test=pd.DataFrame(result_test)
result_train.head()

## Creating a column to store the result
X_train['predict_logReg']=-1
#X_train.drop('predict_logReg',axis=1, inplace=True)
#result_train

## Generating the result from the probs data
for i in range(len(result_train)):
    if (result_train[0][i]>result_train[1][i]):
        X_train.iloc[i,-1]=0
    else:
        X_train.iloc[i,-1]=1



from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
cm_train=confusion_matrix(X_train['predict_logReg'],y_train)
print("Confusion Matrix of Train Dataset")
print(cm_train)
TP=cm_train[1][1]
FP=cm_train[0][1]
FN=cm_train[1][0]
TN=cm_train[0][0]
precision_train=TP/(TP+FP)
Recall_train=TP/(TP+FN)
Accuracy_train=(TP+TN)/(TP+FP+TN+FN)
F1_Score_train=(2*precision_train*Recall_train)/(precision_train+Recall_train)
print("Precision of Train Data",+precision_train)
print("Rrecall of Train Data",+Recall_train)
print("Accuracy of Train Data",+Accuracy_train)
print("F1 Score of Train Data",+F1_Score_train)

print("classification report of Train Dataset")
classificaiton_report_train=classification_report(X_train['predict_logReg'],y_train)
print(classificaiton_report_train)

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


C:\Users\KASTU1\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\KASTU1\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\KASTU1\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Confusion Matrix of Train Dataset
[[3489  669]
 [ 387  737]]
Precision of Train Data 0.5241820768136558
Rrecall of Train Data 0.655693950177936
Accuracy of Train Data 0.8000757288905718
F1 Score of Train Data 0.582608695652174
classification report of Train Dataset
             precision    recall  f1-score   support

          0       0.90      0.84      0.87      4158
          1       0.52      0.66      0.58      1124

avg / total       0.82      0.80      0.81      5282



In [144]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(penalty='l2',C=5)
print(classifier)
result=classifier.fit(X_train, y_train)
print(classifier.score(X_train, y_train))
coefficients = pd.concat([pd.DataFrame(X_train.columns),pd.DataFrame(np.transpose(result.coef_))], axis = 1)
print(coefficients)

print(result)
y_pred_train = result.predict(X_train)
y_pred_test = result.predict(X_test)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
cm_train=confusion_matrix(y_train,y_pred_train)
print("Confusion Matrix of Train Dataset")
print(cm_train)
TP=cm_train[1][1]
FP=cm_train[0][1]
FN=cm_train[1][0]
TN=cm_train[0][0]
precision_train=TP/(TP+FP)
Recall_train=TP/(TP+FN)
Accuracy_train=(TP+TN)/(TP+FP+TN+FN)
F1_Score_train=(2*precision_train*Recall_train)/(precision_train+Recall_train)
print("Precision of Train Data",+precision_train)
print("Rrecall of Train Data",+Recall_train)
print("Accuracy of Train Data",+Accuracy_train)
print("F1 Score of Train Data",+F1_Score_train)

print("classification report of Train Dataset")
classificaiton_report_train=classification_report(y_train,y_pred_train)
print(classificaiton_report_train)

cm_test=confusion_matrix(y_test,y_pred_test)
print("Confusion Matrix of test Dataset")
print(cm_test)
TP=cm_test[1][1]
FP=cm_test[0][1]
FN=cm_test[1][0]
TN=cm_test[0][0]
precision_test=TP/(TP+FP)
Recall_test=TP/(TP+FN)
Accuracy_test=(TP+TN)/(TP+FP+TN+FN)
F1_Score_test=(2*precision_test*Recall_test)/(precision_test+Recall_test)
print("Precision of Test Data",+precision_test)
print("Rrecall of Test Data",+Recall_test)
print("Accuracy of Test Data",+Accuracy_test)
print("F1 Score of Test Data",+F1_Score_test)
classificaiton_report_TEST=classification_report(y_test,y_pred_test)
print(classificaiton_report_train)
#print("classification report of Train Dataset")
#classificaiton_report_test=classification_report(y_test,y_pred_test)
#print(classificaiton_report_test)

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.8000757288905718
                      0         0
0       SeniorCitizen_1  0.273438
1             Partner_1  0.009612
2          Dependents_1 -0.214236
3        PhoneService_1 -0.248267
4     InternetService_1  1.317031
5     InternetService_2 -0.824456
6            Contract_1 -0.781950
7            Contract_2 -1.123472
8    PaperlessBilling_1  0.350892
9       PaymentMethod_1 -0.286361
10      PaymentMethod_2  0.273297
11      PaymentMethod_3  0.031198
12         plan_0000001  0.088891
13         plan_0000010  0.190991
14         plan_0000011  0.048538
15         plan_0000100 -0.131766
16         plan_0000101 -0.060715
17         plan_0000110  0.019447
18         plan_0000111 -0.019373
19         plan_0001000  0.001065
20  

C:\Users\KASTU1\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [150]:
X = fn_dataset.iloc[:,:-1]
Y = fn_dataset.iloc[:,-1:]
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 0)

In [174]:
from sklearn import tree
#clf = tree.DecisionTreeClassifier(min_samples_leaf=50,min_samples_split=50,max_depth=10)
clf = tree.DecisionTreeClassifier()
clf=clf.fit(X_train,y_train)
print(clf)
y_pred_train_p = clf.predict(X_train)
y_pred_test_p = clf.predict(X_test)
y_pred_train_p=pd.DataFrame(y_pred_train_p)
y_pred_test_p=pd.DataFrame(y_pred_test_p)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
cm_train=confusion_matrix(y_train,y_pred_train_p)
print("Confusion Matrix of Train Dataset")
print(cm_train)
print("classification report of Train Dataset")
classificaiton_report_train=classification_report(y_train,y_pred_train_p)
print(classificaiton_report_train)

cm_test=confusion_matrix(y_test,y_pred_test_p)
print("Confusion Matrix of test Dataset")
print(cm_test)
print("classification report of Train Dataset")
classificaiton_report_test=classification_report(y_test,y_pred_test_p)
print(classificaiton_report_test)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
Confusion Matrix of Train Dataset
[[3873    3]
 [  16 1390]]
classification report of Train Dataset
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3876
          1       1.00      0.99      0.99      1406

avg / total       1.00      1.00      1.00      5282

Confusion Matrix of test Dataset
[[1069  229]
 [ 233  230]]
classification report of Train Dataset
             precision    recall  f1-score   support

          0       0.82      0.82      0.82      1298
          1       0.50      0.50      0.50       463

avg / total       0.74      0.74      0.74      1761



In [175]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
%timeit
rf = RandomForestClassifier(min_samples_split=10,n_estimators=80, oob_score=True,max_features=6 ,min_samples_leaf=50, max_depth=7,random_state = 123)
print(rf)
rf.fit(X_train, y_train)
y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

cm_train=confusion_matrix(y_train,y_pred_train)
print("Confusion Matrix of Train Dataset")
print(cm_train)
TP=cm_train[1][1]
FP=cm_train[0][1]
FN=cm_train[1][0]
TN=cm_train[0][0]
precision_train=TP/(TP+FP)
Recall_train=TP/(TP+FN)
Accuracy_train=(TP+TN)/(TP+FP+TN+FN)
F1_Score_train=(2*precision_train*Recall_train)/(precision_train+Recall_train)
print("Precision of Train Data",+precision_train)
print("Rrecall of Train Data",+Recall_train)
print("Accuracy of Train Data",+Accuracy_train)
print("F1 Score of Train Data",+F1_Score_train)
print("classification report of Train Dataset")
classificaiton_report_train=classification_report(y_train,y_pred_train)
print(classificaiton_report_train)

print('************************************************************************************************')

cm_test=confusion_matrix(y_test,y_pred_test)
print("Confusion Matrix of test Dataset")
print(cm_test)
TP=cm_test[1][1]
FP=cm_test[0][1]
FN=cm_test[1][0]
TN=cm_test[0][0]
precision_test=TP/(TP+FP)
Recall_test=TP/(TP+FN)
Accuracy_test=(TP+TN)/(TP+FP+TN+FN)
F1_Score_test=(2*precision_test*Recall_test)/(precision_test+Recall_test)
print("Precision of Test Data",+precision_test)
print("Rrecall of Test Data",+Recall_test)
print("Accuracy of Test Data",+Accuracy_test)
print("F1 Score of Test Data",+F1_Score_test)
print("classification report of Train Dataset")
classificaiton_report_test=classification_report(y_test,y_pred_test)
print(classificaiton_report_test)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features=6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=50, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=1,
            oob_score=True, random_state=123, verbose=0, warm_start=False)


C:\Users\KASTU1\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Confusion Matrix of Train Dataset
[[3876    0]
 [1406    0]]
Precision of Train Data nan
Rrecall of Train Data 0.0
Accuracy of Train Data 0.7338129496402878
F1 Score of Train Data nan
classification report of Train Dataset
             precision    recall  f1-score   support

          0       0.73      1.00      0.85      3876
          1       0.00      0.00      0.00      1406

avg / total       0.54      0.73      0.62      5282

************************************************************************************************
Confusion Matrix of test Dataset
[[1298    0]
 [ 463    0]]
Precision of Test Data nan
Rrecall of Test Data 0.0
Accuracy of Test Data 0.7370812038614424
F1 Score of Test Data nan
classification report of Train Dataset
             precision    recall  f1-score   support

          0       0.74      1.00      0.85      1298
          1       0.00      0.00      0.00       463

avg / total       0.54      0.74      0.63      1761



C:\Users\KASTU1\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\KASTU1\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\KASTU1\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in longlong_scalars


In [176]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(verbose=1, random_state=123, n_estimators=600, learning_rate=0.05,  max_features=6, max_depth = 5, min_samples_leaf=4)
print(gb)
gb.fit(X_train, y_train)
y_pred_train = gb.predict(X_train)
y_pred_test = gb.predict(X_test)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
cm_train=confusion_matrix(y_train,y_pred_train)
print("Confusion Matrix of Train Dataset")
print(cm_train)
TP=cm_train[1][1]
FP=cm_train[0][1]
FN=cm_train[1][0]
TN=cm_train[0][0]
precision_train=TP/(TP+FP)
Recall_train=TP/(TP+FN)
Accuracy_train=(TP+TN)/(TP+FP+TN+FN)
F1_Score_train=(2*precision_train*Recall_train)/(precision_train+Recall_train)
print("Precision of Train Data",+precision_train)
print("Rrecall of Train Data",+Recall_train)
print("Accuracy of Train Data",+Accuracy_train)
print("F1 Score of Train Data",+F1_Score_train)
print("classification report of Train Dataset")
classificaiton_report_train=classification_report(y_train,y_pred_train)
print(classificaiton_report_train)

cm_test=confusion_matrix(y_test,y_pred_test)
print("Confusion Matrix of test Dataset")
print(cm_test)
TP=cm_test[1][1]
FP=cm_test[0][1]
FN=cm_test[1][0]
TN=cm_test[0][0]
precision_test=TP/(TP+FP)
Recall_test=TP/(TP+FN)
Accuracy_test=(TP+TN)/(TP+FP+TN+FN)
F1_Score_test=(2*precision_test*Recall_test)/(precision_test+Recall_test)
print("Precision of Test Data",+precision_test)
print("Rrecall of Test Data",+Recall_test)
print("Accuracy of Test Data",+Accuracy_test)
print("F1 Score of Test Data",+F1_Score_test)
print("classification report of Train Dataset")
classificaiton_report_test=classification_report(y_test,y_pred_test)
print(classificaiton_report_test)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=5,
              max_features=6, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=4, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=600,
              presort='auto', random_state=123, subsample=1.0, verbose=1,
              warm_start=False)
      Iter       Train Loss   Remaining Time 


C:\Users\KASTU1\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


         1           1.1554            2.19m
         2           1.1448            1.10m
         3           1.1297           46.84s
         4           1.1176           35.07s
         5           1.1044           28.01s
         6           1.1024           24.85s
         7           1.0965           21.26s
         8           1.0887           18.57s
         9           1.0845           17.51s
        10           1.0777           15.73s
        20           0.9967            9.54s
        30           0.9624            7.14s
        40           0.9311            6.15s
        50           0.8989            5.40s
        60           0.8765            4.84s
        70           0.8550            4.38s
        80           0.8399            3.96s
        90           0.8274            3.79s
       100           0.8188            3.54s
       200           0.7675            2.18s
       300           0.7447            1.45s
       400           0.7253            0.92s
       500

In [177]:
from numpy import loadtxt
from xgboost import XGBClassifier
XGB = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=10, min_child_weight=30, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=1, reg_lambda=0, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.7,eval_metric='error')
print(XGB)
XGB.fit(X_train, y_train)
y_pred_train = XGB.predict(X_train)
y_pred_test = XGB.predict(X_test)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
cm_train=confusion_matrix(y_train,y_pred_train)
print("Confusion Matrix of Train Dataset")
print(cm_train)
TP=cm_train[1][1]
FP=cm_train[0][1]
FN=cm_train[1][0]
TN=cm_train[0][0]
precision_train=TP/(TP+FP)
Recall_train=TP/(TP+FN)
Accuracy_train=(TP+TN)/(TP+FP+TN+FN)
F1_Score_train=(2*precision_train*Recall_train)/(precision_train+Recall_train)
print("Precision of Train Data",+precision_train)
print("Rrecall of Train Data",+Recall_train)
print("Accuracy of Train Data",+Accuracy_train)
print("F1 Score of Train Data",+F1_Score_train)
print("classification report of Train Dataset")
classificaiton_report_train=classification_report(y_train,y_pred_train)
print(classificaiton_report_train)

cm_test=confusion_matrix(y_test,y_pred_test)
print("Confusion Matrix of test Dataset")
print(cm_test)
TP=cm_test[1][1]
FP=cm_test[0][1]
FN=cm_test[1][0]
TN=cm_test[0][0]
precision_test=TP/(TP+FP)
Recall_test=TP/(TP+FN)
Accuracy_test=(TP+TN)/(TP+FP+TN+FN)
F1_Score_test=(2*precision_test*Recall_test)/(precision_test+Recall_test)
print("Precision of Test Data",+precision_test)
print("Rrecall of Test Data",+Recall_test)
print("Accuracy of Test Data",+Accuracy_test)
print("F1 Score of Test Data",+F1_Score_test)
#print("classification report of Train Dataset")
#classificaiton_report_test=classification_report(y_test,y_pred_test)
#print(classificaiton_report_test)

X_test['pred']=y_pred_test

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, eval_metric='error', gamma=0,
       learning_rate=0.2, max_delta_step=0, max_depth=10,
       min_child_weight=30, missing=None, n_estimators=300, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=1, reg_lambda=0, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.7)


C:\Users\KASTU1\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\KASTU1\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\KASTU1\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\KASTU1\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this w

Confusion Matrix of Train Dataset
[[3560  316]
 [ 551  855]]
Precision of Train Data 0.7301451750640479
Rrecall of Train Data 0.6081081081081081
Accuracy of Train Data 0.8358576296857251
F1 Score of Train Data 0.6635622817229337
classification report of Train Dataset
             precision    recall  f1-score   support

          0       0.87      0.92      0.89      3876
          1       0.73      0.61      0.66      1406

avg / total       0.83      0.84      0.83      5282

Confusion Matrix of test Dataset
[[1149  149]
 [ 231  232]]
Precision of Test Data 0.6089238845144357
Rrecall of Test Data 0.5010799136069114
Accuracy of Test Data 0.7842135150482681
F1 Score of Test Data 0.5497630331753555


C:\Users\KASTU1\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
